# Welcome to the PrecisionTrack Sanity Check Notebook

![alt text](https://raw.githubusercontent.com/VincentCoulombe/precision_track/main/assets/logo.png)

### In this notebook, you will:
- Deploy a fresh PrecisionTrack system, tailored for the Colab's provided hardware.  
- Track and analyze the behaviors of mice from the provided sanity check footage.  
- Run tracking on the **MICE dataset** validation video.  
- Visualize the results directly in the notebook.  


**Note:**  
To train your own PrecisionTrack checkpoints, please refer to our [tutorials](https://github.com/VincentCoulombe/precision_track/tree/main) and [configuration documentation](https://github.com/VincentCoulombe/precision_track/tree/main/configs).


### Before you begin
Ensure your Colab runtime is connected to a GPU:

1. Click on **Runtime** in the menu bar.  
2. Select **Change runtime type**.  
3. Set the interpreter to **Python 3**.  
4. Under **Hardware accelerator**, select **GPU**.  

In [ ]:
import subprocess

# First, determine if the machine is cuda accelerated (to determine which version of the virtual environment we are goind to build).

try:
    CUDA_ACCELERATED = subprocess.run("nvidia-smi").returncode == 0
except FileNotFoundError:
    CUDA_ACCELERATED = False
    print("Please follow the instructions in the cell above to CUDA-accelerate your instance.")

In [ ]:
%%capture
# Second, build PrecisionTrack's virtual environment.
!git clone https://github.com/VincentCoulombe/precision_track.git
%cd /content/precision_track/

if CUDA_ACCELERATED:
    !pip install torch==2.7.1 torchvision==0.22.1  --index-url https://download.pytorch.org/whl/cu128
    !pip install -e .[cuda]
else:
    !pip install torch==2.7.1 torchvision==0.22.1  --index-url https://download.pytorch.org/whl/cpu
    !pip install -e .[cpu]
    
!pip install gdown
# NOTE: This took some time when I tried it (about 12 minutes). This is because Pytorch and PyCuda are both heavy, wheeled, depedencies.  

In [ ]:
# This cell contains a custom helper function for visualizing video on the browser.
from IPython.display import HTML
from base64 import b64encode
import subprocess


def to_h264(video_path: str) -> tuple:
    """Convert the video codec to a browser-friendly one."""
    subprocess.run(
        [
            "ffmpeg",
            "-y",
            "-i",
            video_path,
            "-c:v",
            "libx264",
            "-profile:v",
            "baseline",
            "-level",
            "3.0",
            "-pix_fmt",
            "yuv420p",
            "-movflags",
            "+faststart",
            "./visualization_h264.mp4",
        ],
        check=True,
    )

    mp4 = open("./visualization_h264.mp4", "rb").read()
    return "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
from pathlib import Path
import os

# Third, download the provided checkpoints and change the "deployed_directory" setting to a predefined directory.

!mkdir -p /content/precision_track/checkpoints/model_mice/
%cd /content/precision_track/checkpoints/model_mice/


!gdown --id 17Up0YDRjNQp3mI75dKPtLPnIGDXkkCgm
!gdown --id 1yr0CC0_EAZEBjqkzzGOULbTH_8wQ6bq1

# NOTE: This step is done Programmatically here, but I encourage the users to do it manually. Please refer to our settings and workflow guides for more details.
file = Path(os.path.abspath("/content/precision_track/configs/settings/mice.py"))
text = file.read_text()
text = text.replace("../tests/configs/", "/content/precision_track/checkpoints/model_mice/") # Change the deployment directory
text = text.replace("../../datasets/mice/", "/content/precision_track/assets/")
text = text.replace("images/0000003435.jpg", "0000004668.jpg")

file.write_text(text)

In [ ]:
import re
from mmengine.logging import print_log
from logging import WARNING

# Fourth, deploy the pretrained PrecisionTrack's system. NOTE: Here we do not perform hyperparameter optimisation as they are already cached. This saves you some time.
%cd /content/precision_track/tools/

!python ./deploy.py false

# Next, we'll register the newly deployed engines in our settings file.
# NOTE: This step is done Programmatically here, but I encourage the users to do it manually instead (its way more intuitive). Please refer to our settings and workflow guides for more details.
file = Path("/content/precision_track/configs/settings/mice.py")
text = file.read_text()

# Dynamically update the engine settings.
deployed_checkpoints = os.listdir("/content/precision_track/checkpoints/model_mice/")
default_runtimes = ["model_mice_clustering_DEPLOYED.onnx", "mart_DEPLOYED.onnx"]
patterns = [r"^model.*\.engine$", r"^mart.*\.engine$"]
engines = [None, None]

for deployed_checkpoint in deployed_checkpoints:
    for i, pattern in enumerate(patterns):
        if re.search(pattern, deployed_checkpoint) is not None:
            engines[i] = os.path.basename(deployed_checkpoint)

for engine, default_runtime in zip(engines, default_runtimes):
    if engine is None:
        print_log(f"The deployment was not sucessfull. falling back to the default runtime: {default_runtime}", logger="current", level=WARNING)
    else:
        print_log(f"The following engine: '{engine}' is now available for you to download and possibly re-use in futur Colab instances.", logger="current")
        text = text.replace(default_runtime, engine) # Point to the newly deployed engine

file.write_text(text)

# We are ready to go! 

We'll begin by tracking on the small sanity check video [provided with the repository](https://github.com/VincentCoulombe/precision_track/blob/main/assets/20mice_sanity_check.avi).

### Before we start
- We will use a pretrained checkpoint from the **MICE dataset**.  
    - This checkpoint was optimized for a specific camera angle.  
    - Using it on footage with different viewpoints and/or other species will likely produce nonsensical results.  
    - To train a custom PrecisionTrack model for your own footage, please refer to the **"Where to start?"** section of our [README](https://github.com/VincentCoulombe/precision_track/tree/main).

### Limitations when running on Colab
- **Single-core execution:**  
  Colab typically provides a single-core CPU. This disables our multi-processing pipelines.  
  - For example, tracking and action recognition normally run in parallel on separate cores.  
  - On modern local machines (e.g., mine with 8 cores), these processes execute simultaneously.  

- **JiT compilation overhead:**  
  Most heavy computations are **Just-in-Time (JiT) compiled**, which adds a fixed startup cost for each tracking session.  
  - On longer videos, this overhead quickly becomes negligible.  
  - To see this effect, you can also try tracking on our larger [validation video](https://drive.google.com/file/d/1z2PF7UlNBF2PudS3UP-cGejBIttbpZBm/view?usp=drive_link).

In [ ]:
# First, Track via PrecisionTrack's tracking tool (yes, it is that simple).
!python ./track.py ../assets/20mice_sanity_check.avi

In [ ]:
# Second, run PrecisionTrack's visualization tool.

!python ./visualize.py ../assets/20mice_sanity_check.avi ./sanity_check_visualization.mp4

# This will create the "./sanity_check_visualization.mp4" video. This video can be displayed by running this notebook's second to last cell. 

### Tracking on a longer video (without action recognition)

To better demonstrate PrecisionTrack’s inference speed, we will track on a longer video **without inferring actions**.  


#### Steps:
1. Set `"with_action_recognition"` to **False** in the configuration file (**Done in the very next cell**).  
   - You can refer to the [settings guide](https://github.com/VincentCoulombe/precision_track/tree/main/configs) for more details.  

2. Download the longer [validation video](https://drive.google.com/file/d/1z2PF7UlNBF2PudS3UP-cGejBIttbpZBm/view?usp=drive_link) from the MICE sequential dataset (**Done one cell further down**).  


In [ ]:
# First, change the "with_action_recognition" setting from "True" to "False".
file = Path("../configs/settings/mice.py")
text = file.read_text()
text = text.replace("with_action_recognition = True", "with_action_recognition = False")
file.write_text(text)

In [ ]:
# Second, download the MICE dataset's validation split.
!gdown --id 1z2PF7UlNBF2PudS3UP-cGejBIttbpZBm

In [ ]:
# Third, rerun PrecisionTrack's tracking tool (now on the full 5 minutes validation footage).

!rm -r ../work_dir/ # Remove previous results.

!python ./track.py ./14-20-02.avi

# NOTE 1: Although the FPS increased, the gain is still far below my expectations.

# NOTE 2: Even when using the expensive 'A100' GPU option, tracking was considerably slower 
#         than on the several cheaper machines we are currently running PrecisionTrack on. 
#         I suspect this slowdown may be due to underclocked CPUs. 

# NOTE 3: After experimenting further, I conclude that the best option to reproduce our 
#         latency results is to run PrecisionTrack locally on a machine matching the
#         specifications listed in the publication's "Computational Resource Usage" section. 
#         Building such a workstation (a clone of my personal setup) should cost around $1.5k.
#         To install locally, please refer to our Installation guide. 

In [ ]:
# Fourth, run PrecisionTrack's visualization tool.

!python ./visualize.py ./14-20-02.avi ./val_split_visualization.mp4

# This will create the "./val_split_visualization.mp4" video. This video can be displayed by running this notebook's last cell. 

# NOTE: For some reason, the same exact rendering process is about 10x slower here than on our local machines.

### Visualization

Run the next two cells to display the predictions on the sanity check footage and the validation footage, respectively.


In [ ]:
# Display the sanity check video.
data_url = to_h264("./sanity_check_visualization.mp4")

HTML(
    f"""
<video width=800 controls>
    <source src="{data_url}" type="video/mp4">
</video>
"""
)

In [ ]:
# Display the validation footage video.
data_url = to_h264("./val_split_visualization.mp4")

HTML(
    f"""
<video width=800 controls>
    <source src="{data_url}" type="video/mp4">
</video>
"""
)